<a href="https://colab.research.google.com/github/slayerzeroa/Railway_Accident_BERT/blob/main/railway_accident_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment Setting
+ Installing
+ Call Library
+ Set Cuda
+ Google Api Key

In [2]:
!pip install transformers
!pip install nltk
!pip install pytorch-crf
!pip install konlpy
!pip install collections

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 31.3 MB/s 
     |████████████████████████████████| 182 kB 65.0 MB/s 
     |████████████████████████████████| 7.6 MB 16.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 6.7 MB/s 
     |████████████████████████████████| 465 kB 68.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement collections (from versions: none)
ERROR: No matching distribution found for collections


In [3]:
# 라이브러리
import torch
from torch import nn, optim, tensor
from torch.utils.data import DataLoader, Dataset,TensorDataset, random_split
import torch.nn.functional as F
from torchcrf import CRF

import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, mean_squared_error

from transformers import BertTokenizer, BertForMaskedLM


import json
import glob
import re
import os
import ast
import html

import re

import nltk

from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize

from konlpy.tag import Okt
from collections import Counter

from google.colab import files

In [4]:
#punkt 다운로드드
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
#디바이스 세팅
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
# GOOGLE API KEY: json
key = files.upload()

#Tokenizer
-> 목적에 알맞는 토크나이저로 튜닝

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [8]:
def tokenizing_(token_list, label_list, tokenizer):
  text_seq = []
  valid_seq = []
  label_seq = label_list
  for tokens, label in zip(token_list, label_list):
    new_target_text = []
    new_valid = []
    for i, (t, l) in enumerate(zip(tokens, label)):
      tokens_wordpiece = tokenizer.tokenize(t)
      new_v = [1] + [0]*(len(tokens_wordpiece)-1)
      new_target_text.extend(tokens_wordpiece)
      new_valid.extend(new_v)
    valid_seq.append(new_valid)
    text_seq.append(new_target_text)
  return text_seq, valid_seq, label_seq

In [ ]:
text_seq, valid_seq, label_seq = tokenizing_(token_list, label_list, tokenizer)

In [ ]:
def tokenizing_rel(token_list, label_list, rel_list, tokenizer):
  text_seq = []
  valid_seq = []
  final_left_seq = []
  final_right_seq = []
  label_seq = []

  for tokens, label, rel in zip(token_list, label_list, rel_list):
    lefts, rights, labels = rel

    for le, ri, la in zip(lefts, rights, labels):
      new_target_text = []
      new_valid = []
      new_left = []
      new_right = []
      before_label = -1

      for i, (t, l) in enumerate(zip(tokens, label)):
        tokens_wordpiece = tokenizer.tokenize(t)
        if before_label != l and l != -1:
          if l == le:
            tokens_wordpiece = ["*"] + tokens_wordpiece
          elif l == ri:
            tokens_wordpiece = ["#"] + tokens_wordpiece
        
        if l != -1 and i+1 != len(label) and l != label[i+1]:
          if l == le:
            tokens_wordpiece = tokens_wordpiece + ["*"]
          elif l == ri:
            tokens_wordpiece = tokens_wordpiece + ["#"]
        
        if l != -1 and i+1 == len(label):
          if l == le:
            tokens_wordpiece = tokens_wordpiece + ["*"]
          elif l == ri:
            tokens_wordpiece = tokens_wordpiece + ["#"]
        
        before_label = l
        new_v = [l]*(len(tokens_wordpiece))
        new_target_text.extend(tokens_wordpiece)
        new_valid.extend(new_v)
      
      leftaa = [float(x == le) for x in new_valid]
      rightaa = [float(x == ri) for x in new_valid]
      final_left_seq.append(leftaa)
      final_right_seq.append(rightaa)
      text_seq.append(new_target_text)
      label_seq.append(la)
    return text_seq, final_left_seq, final_right_seq, label_seq

In [ ]:
def make_greedy_label(rel_list, dic_rel_label, max_labels):
  final_rel_list = []

  for rel, max_lab in zip(rel_list, max_labels):
    temp_les = []
    temp_re =[]
    rel_labels = rel[4]
    rel_lefts = rel[5]
    rel_rights = rel[6]
    max_value = max_lab
    l, r = make_fair_by_max(max_value)
    new_l = []
    new_r = []
    qomax = len(rel_labels)
    z = 0

    for ll, rr in zip(l, r):
      q = 0
      laba = 0
      for lf, ri, rel_label in zip(rel_lefts, rel_rights, rel_labels):
        if ll == lf and rr == ri:
          q = 1
          laba = dic_rel_label[rel_label]
      
      if q == 1:
        new_l.append(ll)
        new_r.append(rr)
        temp_les.append(laba)
      elif z < 1.5*qomax:
        new_l.append(ll)
        new_r.append(rr)
        z = z + 1
        temp_les.append(0)
      
    final_rel_list.append((new_l, new_r, temp_les))
  return final_rel_list

#BERT Model Fine Tuning by MLM

In [12]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [7]:
# Dataset: plain.html
class MLMTrainDataset_Sik(Dataset):
  
  def __init__(self, max_len, mask_prob, mask_token, sep_token, cls_token, all_sents,
               tokenizer, pad_token):
    self.all_sents = all_sents
    self.num_vocabs = tokenizer.vocab_size
    self.max_len = max_len
    self.mask_prob = mask_prob
    self.mask_token = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(mask_token))
    self.cls_token = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(cls_token))
    self.sep_token = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sep_token))
    self.tokenizer = tokenizer
    self.pad_token = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(pad_token))
  
  def __len__(self):
    return len(self.all_sents)
  
  def __getitem__(self, index):
    sents = self.all_sents[index]
    sents = self.tokenizer.convert_tokens_to_ids(self.tokenizer.tokenize(sents))
    tokens = []
    sents = np.array(sents)
    inputs = sents
    labels = inputs.copy()

    probability_matrix = torch.full(labels.shape, self.mask_prob)

    masked_indices = torch.bernoulli(probability_matrix).bool()
    
    labels[~masked_indices] = -100

    indices_replaced = torch.bernoulli(torch.full(labels.shape, 0.8)).bool() & masked_indices

    inputs[indices_replaced] = self.tokenizer.convert_tokens_to_ids(self.mask_token)

    indices_random = torch.bernoulli(torch.full(labels.shape, 0.5)).bool() & masked_indices & ~indices_replaced

    random_words = torch.randint(len(self.tokenizer), labels.shape, dtype = torch.long)

    inputs[indices_random] = random_words[indices_random]
    inputs = list(inputs)
    labels = list(labels)
    valid_length = min(len(inputs), self.max_len-2)
    tokens = self.cls_token + inputs[:self.max_len-2] + self.sep_token
    print("tokens")
    print(tokens)
    labels = [-100] + labels[:self.max_len-2] + [-100]

    mask_len = self.max_len - len(tokens)

    tokens = tokens + self.pad_token * mask_len
    labels = labels + [-100] * mask_len
    attention_masks = [0.0]*len(tokens)
    for i in range(valid_length):
      attention_masks[i] = 1.0
    
    return torch.LongTensor(tokens), torch.LongTensor(labels), valid_length, torch.LongTensor(attention_masks)

In [8]:
sample = files.upload()

Saving sample.plain.html to sample.plain.html


In [9]:
# sample.plain.html: any plain.html
f = open('sample.plain.html')
read = f.read()
print(read)
print(type(read))

<!DOCTYPE html >
<html id="a3DnzcVSvJKG25OeznRpiW5zLgi8-sample.pdf" data-origid="sample.pdf" class="anndoc" data-anndoc-version="3.6" lang="" xml:lang="" xmlns="http://www.w3.org/1999/xhtml">
  <head>
    <meta charset="UTF-8"/>
    <meta name="generator" content="net.tagtog.anndoc.v3.parsers.general.pdf.common.StubPdfParser_v1_0_0"/>
    <title>a3DnzcVSvJKG25OeznRpiW5zLgi8-sample.pdf</title>
  </head>
  <body>
    <article>
      <section data-type="">
        <div class="content">
          <p id="s1p1">보고서 번호: ARAIB/R 2020 - 3 철도사고 조사보고서</p>
          <p id="s1p2">한국철도공사 경전선(완사역→진주역 사이, 삼랑진역 기점 103.286km) 제3081화물열차(디젤 7620호 + 화차 25량) 열차탈선 2019년 2월 18일(월) 21시 56분경 2020.</p>
          <p id="s1p3">6.</p>
          <p id="s1p4">19.</p>
          <p id="s1p5">항공･철도사고조사위원회 이 조사보고서는 「항공․철도사고조사에 관한 법률」제 2조에 의거 사고조사가 이루어졌으며, 제25조에 따라 작 성되었다. 같은 법률 제1조에서 「철도사고 조사는 독립적이고 공정 한 조사를 통하여 사고 원인을 정확하게 규명함으로써 철도 사고의 예방과 안전 확보에 이바지함」을 목적으로 하고 있 다. 또한, 제30조에 따라 사고조사는 민․형사상 책임과 관련된 사 법절차, 행정처분 절차 또는 행정

In [10]:
check = re.sub('(<([^>]+)>)', '', read).strip()
check = re.sub('\n', '', check)
check = re.sub(' +', ' ', check)
check = check[40:]
check = re.sub('·','', check)
check = re.sub('○', '', check)
print(check)

보고서 번호: ARAIB/R 2020 - 3 철도사고 조사보고서 한국철도공사 경전선(완사역→진주역 사이, 삼랑진역 기점 103.286km) 제3081화물열차(디젤 7620호 + 화차 25량) 열차탈선 2019년 2월 18일(월) 21시 56분경 2020. 6. 19. 항공･철도사고조사위원회 이 조사보고서는 「항공․철도사고조사에 관한 법률」제 2조에 의거 사고조사가 이루어졌으며, 제25조에 따라 작 성되었다. 같은 법률 제1조에서 「철도사고 조사는 독립적이고 공정 한 조사를 통하여 사고 원인을 정확하게 규명함으로써 철도 사고의 예방과 안전 확보에 이바지함」을 목적으로 하고 있 다. 또한, 제30조에 따라 사고조사는 민․형사상 책임과 관련된 사 법절차, 행정처분 절차 또는 행정 쟁송 절차와 분리․수행되어 야 하고, 제32조에서위원회에 진술․증언․자료 등의 제출 또는 답변을 한 사람은 이를 이유로 해고․전보․징계․부당한 대우 또 는 그 밖에 신분이나 처우와 관련하여 불이익을 받지 아니 한다.라고 규정하고 있다. 그러므로 이 조사보고서는 철도 분야의 안전을 증진시킬 목 적 이외의 용도로 사용되어서는 아니 된다. 차례 한국철도공사 경전선 완사역∼진주역간 화물열차 탈선사고 조사보고서 - i - 차 례 한국철도공사 경전선 완사역∼진주역간 화물열차 탈선사고  1 개요  2 1. 사실 정보  3 1.1 사고의 경위  3 1.2 피해 사항  4 1.3 인적 정보 및 업무 수행사항  4 1.4 현장 정보  10 1.5 열차와 차량 정보  11 1.6 차량정비단의 차축 베어링 분해 검수 실태  16 1.7 신호 및 전차선 정보  18 1.8 기상 정보  19 2. 분석  20 2.1 업무 수행사항 분석  20 2.2 차축 베어링 분해 검사  22 2.3 컨테이너 상하차 작업 분석  25 2.4 그리스 성분 분석  26 2.5 종합 분석  29 3. 결론  30 3.1 조사 결과  30 3.2 사고 원인  32 4. 안전 권고  33 4.1 한국철도공사에 대하여  33 제목 한국

In [13]:
# set token
sep_token="[SEP]"
pad_token="[PAD]"
cls_token="[CLS]"
mask_token="[MASK]"
max_len = 60
mask_prob = 0.15

# make instance
test_instance = MLMTrainDataset_Sik(max_len, mask_prob, mask_token, sep_token, cls_token, check, bert_tokenizer, pad_token)

In [14]:
# check instance
print(test_instance[2:100])

tokens
[101, 9425, 9338, 20309, 131, 50884, 10738, 100, 120, 155, 23607, 118, 124, 87624, 12945, 11664, 9678, 12945, 100, 11664, 12424, 48556, 80012, 28000, 12945, 8885, 16617, 18471, 113, 9591, 12945, 23160, 111675, 18623, 16323, 23160, 9405, 10739, 117, 9410, 62200, 100, 23160, 8932, 34907, 15553, 100, 28747, 23440, 100, 9672, 32792, 11396, 10759, 18227, 100, 100, 23466, 113, 102]
(tensor([   101,   9425,   9338,  20309,    131,  50884,  10738,    100,    120,
           155,  23607,    118,    124,  87624,  12945,  11664,   9678,  12945,
           100,  11664,  12424,  48556,  80012,  28000,  12945,   8885,  16617,
         18471,    113,   9591,  12945,  23160, 111675,  18623,  16323,  23160,
          9405,  10739,    117,   9410,  62200,    100,  23160,   8932,  34907,
         15553,    100,  28747,  23440,    100,   9672,  32792,  11396,  10759,
         18227,    100,    100,  23466,    113,    102]), tensor([ -100,  -100,  -100,  -100,  -100,  -100,  -100, 72918,  -100,  -10

<ipython-input-7-8761e6fd77f8>:58: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return torch.LongTensor(tokens), torch.LongTensor(labels), valid_length, torch.LongTensor(attention_masks)


# BERT Relation

In [30]:
# 개체명 인식 모델 Class 코드
class Bert_entity(nn.Module):
  def __init__(self, model, num_labels):
    super().__init__()
    self.model = model
    self.hidden_size = 768
    self.block_size = 64
    self.hidden_layer = nn.Linear(768, 768)
    self.classifier = nn.Linear(768, num_labels)
    self.dropout = nn.Dropout(0.4)
    self.num_labels = num_labels
    self.crf = CRF(num_tags = num_labels, batch_first=True)
  
  def forward(self,
              input_ids = None,
              attention_mask = None,
              labels = None,
              valid_ids = None,
              label_mask = None):
    
    output = self.model(input_ids = input_ids, attention_mask = attention_mask)
    last_hidden_state = output.last_hidden_state
    valid_output = torch.zeros(last_hidden_state.size()[0], last_hidden_state.size()[1],
                               last_hidden_state.size()[2], dtype=torch.float32, device='cuda:1')
    valid_masks = torch.zeros(last_hidden_state.size()[0], last_hidden_state.size()[1],
                              dtype=torch.bool, device='cuda:1')
    
    for i in range(last_hidden_state.size()[0]):
      jj = -1
      for j in range(last_hidden_state.size()[1]):
        if valid_ids[i][j].item() == 1:
          jj += 1
          valid_masks[i][jj] = 1
          valid_output[i][jj] = last_hidden_state[i][j]
      
      relations = self.dropout(valid_output)
      logits = self.classifier(relations)
      loss = None
      if labels is not None:
        if attention_mask is not None:
          loss, logits = self.crf(logits, labels, mask = valid_masks, reduction = 'token_mean'), self.crf.decode(logits, mask=valid_masks)
        else:
          loss, logits = self.crf(logits, labels, mask = valid_masks, reduction = 'token_mean'), self.crf.decode(logits, mask=valid_masks)
      return -loss, logits, last_hidden_state, valid_output, output.attentions


In [31]:
# 개체간 관계 추출 모델 Class 코드
class Bert_relation(nn.Module):
  def __init__(self, model, num_labels):
    super().__init__()
    self.model = model
    self.hidden_size =768
    self.block_size = 64

    self.head_extractor = nn.Linear(768, 768)
    self.tail_extractor = nn.Linear(768, 768)

    self.bilinear_classifier = nn.Linear(768 * 3, num_labels)

    self.dropout = nn.Dropout(0.1)
    self.num_labels = num_labels
  
  def get_hrts(self, valid_output, left_ids, right_ids):
    left_masks = left_ids.unsqueeze(-1).expand(valid_output.size()).float()
    right_masks = right_ids.unsqueeze(-1).expand(valid_output.size()).float()

    left_rep = torch.logsumexp(valid_output * left_masks, dim=1)
    right_rep = torch.logsumexp(valid_output * right_masks, dim=1)

    return left_rep, right_rep
  
  def forward(self,
              input_ids=None,
              attention_mask=None,
              labels=None,
              valid_ids=None,
              label_mask=None,
              left_ids=None,
              right_ids=None):
    output = self.model(input_ids=input_ids, attention_mask=attention_mask)
    last_hidden_state = output.last_hidden_state
    pooled_output = output.pooler_output
    context = self.dropout(pooled_output)
    valid_output = last_hidden_state

    left_rep, right_rep = self.get_hrts(valid_output, left_ids, right_ids)

    left_rep = self.head_extractor(left_rep)
    right_rep = self.head_extractor(right_rep)

    bl = torch.cat([context, left_rep, right_rep], dim=-1)

    logits = self.bilinear_classifier(bl)
    loss = None

    if labels is None:
      loss_fct = torch.nn.BCEWithLogitsLoss()
      loss = loss_fct(logits, labels.float())

    output = logits

    return loss, output if loss is not None else output

In [32]:
def find_most_similar_words(word, total_words):
  q = []

  for a in total_words:
    k = jaccard_similarity(word, a[0])
    q.append((a, k))
  q.sort(key=lambda x:x[1])
  qq = q[-20:]
  new_q = []
  for qa in qq:
    if qa[1] > 0.05:
      new_q.append(qa)
  
  return new_q

In [33]:
def jaccard_similarity(word1, word2):
  word1 = word1.lower()
  word2 = word2.lower()
  trigram1 = trigram_from_word(word1)
  trigram2 = trigram_from_word(word2)

  set_trigram = set(trigram1)
  set_trigram = set_trigram.union(set(trigram2))
  con_trigram = [t in trigram2 for t in trigram1]
  qq = sum(con_trigram)
  la = 0.0
  la = float(float(qq)/float(len(set_trigram)+1e-9))

  return la

In [34]:
def trigram_from_word(word):
  if len(word) < 3:
    return [word]
  
  else:
    trigram = [word[i:i+3] for i in range(len(word)-3)]
    return trigram

#Tokenizing

In [ ]:
str_ = output_string.getvalue()
new_str = re.sub("\n", "", str_)
new_str = re.sub("·", "", new_str)

In [ ]:
word_tokens = word_tokenize(new_str)
sent_tokens = sent_tokenize(new_str)

In [ ]:
okt = Okt()
noun = okt.nouns(text)
for i, v in enumerate(noun):
  if len(v) < 2:
    noun.pop(i)

count = Counter(noun)

In [ ]:
# 단어 빈도 카운트
noun_list = count.most_common(100)
for v in noun_list:
  print(v)

#BERT Training

In [10]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForMaskedLM.from_pretrained('bert-base-multilingual-cased')

Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [35]:
inputs = bert_tokenizer(check, return_tensors='pt')

Token indices sequence length is longer than the specified maximum sequence length for this model (30434 > 512). Running this sequence through the model will result in indexing errors


{'input_ids': tensor([[  101,  1037, 29097,  ...,  1047,  2099,   102]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}


In [36]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [38]:
rand = torch.rand(inputs.input_ids.shape)

mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * (inputs.input_ids != 102) # 101, 102번 토큰 제외하고 15% 위치 선별

### padding이 포함된 경우(0번도 추가 제외) ###
# mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * (inputs.input_ids != 102) * (inputs.input_ids != 0)

selection = torch.flatten((mask_arr[0]).nonzero())

In [39]:
import numpy as np

selection_val = np.random.random(len(selection)) # selection의 위치마다 0~1 값 부여

mask_selection = selection[np.where(selection_val >= 0.2)[0]] # 80% : Mask 토큰 대체
random_selection = selection[np.where(selection_val < 0.1)[0]] # 10% : 랜덤 토큰 대체

print(random_selection) # tensor([ 30,  95, 143])
print(mask_selection)

tensor([   35,    43,   143,   208,   262,   266,   321,   374,   479,   481,
          572,   698,   710,   832,   854,   866,   996,  1023,  1047,  1070,
         1194,  1324,  1367,  1391,  1438,  1450,  1463,  1501,  1665,  1681,
         1741,  1757,  1857,  1965,  1988,  1990,  2104,  2111,  2132,  2299,
         2353,  2389,  2402,  2652,  2679,  2719,  2814,  2871,  2987,  3132,
         3221,  3273,  3280,  3337,  3547,  3565,  3588,  3615,  3672,  3714,
         3720,  3739,  3827,  3885,  3893,  3947,  3963,  4000,  4017,  4033,
         4077,  4090,  4136,  4190,  4247,  4257,  4317,  4428,  4433,  4465,
         4486,  4537,  4548,  4557,  4579,  4618,  4632,  4650,  4855,  4879,
         5007,  5068,  5364,  5455,  5544,  5554,  5646,  5681,  5798,  5924,
         5937,  5968,  6091,  6190,  6297,  6407,  6543,  6570,  6581,  6592,
         6597,  6734,  6868,  6889,  6908,  6927,  7049,  7065,  7180,  7191,
         7242,  7324,  7355,  7391,  7425,  7428,  7537,  7766, 

In [40]:
inputs.input_ids[0, mask_selection] = 103
inputs.input_ids[0, random_selection] = torch.randint(0, 30522, size = random_selection.shape)

print(inputs)

{'input_ids': tensor([[  101,  1037, 29097,  ...,  1047,  2099,   102]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([[  101,  1037, 29097,  ...,  1047,  2099,   102]])}


In [ ]:
outputs = model(**inputs)